# PROYECTO II  PROGRAMA DE FORMACIÓN  MLDS AVANZADO
## Daniel F. Benavides R. 
## Módulo V - Deep Learning

### OBJETIVO

El objeto del presente proyecto es poner en producción las técnicas de Deep Learning exploradas en el presente módulo.  La empresa para la cual trabajo  se encarga de la supervisión del servicio de transporte en la ciudad;  la operación es realizada por terceros y sería un gran avance  poder automatizar algunas labores que viene siendo realizada manualmente porque para las mismas no se tiene suficiente personal. Es por esto  que se plantea la utilización de un modelo de _**clasificación de audio**_ de la librería **_transformers_** a partir del cual aplicando **_Fine Tuning_** y que de ello pueda derivar algún producto con el cual se automaticen tareas de supervisión de la operación. 


Con esta idea inicia el presente, sin embargo luego de la verificación de trabajos como el realizado por  [Julien Simon](https://huggingface.co/juliensimon) quien en un ejercicio de Fine Tuning a un modelo de Facebook [facebook/wav2vec2-conformer-rel-pos-large](https://huggingface.co/facebook/wav2vec2-conformer-rel-pos-large) que realiza tarea de Automatic Speech Recognition logra llevarlo a un modelo de  Audio Classification  aplicando la técnica de  fine_tuning con el dataset _speech commands_ [juliensimon/wav2vec2-conformer-rel-pos-large-finetuned-speech-commands](https://huggingface.co/juliensimon/wav2vec2-conformer-rel-pos-large-finetuned-speech-commands). En este trabajo se encuentra que esta una meta muy alta para ser la primera vez que se hace una aproximación a un ejercicio real de *Fine Tuning*, pues requiere insumos muy complejos como un dataset muy depurado para reentrenar el modelo  además demanda gastos computacionales como el mismo autor relacionado lo explica  en [Audio Classification with Hugging Face Transformers](https://www.youtube.com/watch?v=iuvDLKql3yk)


Habiendo explorado las bondades  de la técnica de fine tuning y  conocido más de fondo la poderosa herramienta que es  reutilizar modelos pre-entrenados para otras labores por medio del reentranamiento del cabezal;  decido tomar un modelo de clasificación de sentimientos y utilizarlo para hacer la clasificación de **SMS** no deseado por medio de la aplicación de Fine Tuning. Este ejercicio fue realizado con base en el siguiente vídeo [Fine Tuning Pretrained Model On Custom Dataset Using 🤗 Transformer
](https://www.youtube.com/watch?v=V1-Hm2rNkik).

Debido a que el objetivo del  proyecto es la aplicación  los contenidos vistos en el módulo; se tomó este ejercicio de baja complejidad como base para entender más de fondo la técnica de Fine Tuning con miras a que en próximas oportunidades y supliendo los recursos necesarios, esta técnica  se pueda llevar al plano productivo  (inicialmente planteado). Del módulo V queda la fascinación por estas dos técnicas, lo que fija como objetivo personal seguir explorando su utilización y las posibes formas de aplicación en mi labor profesional. 

# Carga de datos 

A continuación importamos pandas, por medio del cual hacemos el respectivo cargue del dataset, delimitamos por el espacio la  etiqueta del mensaje.

Luego por medio de la función _list_ convertimos el mensaje y las etiquetas en un par de listas. luego convertimos las etiquetas en una variable dummie, debido a que tenemos una salida binaria  _(el mensaje es spam o no lo es)_

In [46]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])
X=list(df['message'])
y=list(df['label'])
y=list(pd.get_dummies(y,drop_first=True)['spam'])

### Preprocesamiento 

Ahora importamos la función *train_test_split*  del módulo *model_selection* de la librería *scikit-learn*  y por medio de este dividimos  en set de entrenamiento y prueba. Definimos el tamaño de set de prueba en 20% de la muestra. También definimos el parámetro *random_state* para efectos de controlar la generación  de los dos conjuntos de tal manera que  no sean aleatorios. 

Luego instalamos la librería transformers, aunque en mi caso ya lo había realizado. 


In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [31]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Ahora debemos invocar los modelos de que vamos a utilizar de la librería transformers  en los siguientes pasos: 

* Llamamos el modelo preentrenado
* Llamamos el tokenizador 

Necesitamos aplicar el tokenizador sobre nuestro conjunto de datos. 

Así que acontinuación llamamos de la librería transformers el tokenizador _"DistilBertTokenizerFast"_ luego lo definimos como nuestro **tokenizer** indicando que el mismo proviene del modelo preentrenado [_'distilbert-base-uncased'_](https://huggingface.co/distilbert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.)

In [32]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Luego  aplicamos el tokenizador que acabamos de definir sobre nuestro conjunto  de mensajes de entrenamiento y prueba. Como los SMS no tienen la misma longitud (cantidad de tokens) debemos definir los parámetros truncation y padding como True para que se obtener oraciones del mismo tamaño; uno se encarga de rellenar de ceros (padding) y el otro de truncar las oraciones más largas. Esto para obtener un conjunto y luego tensores rectangulares. 

In [33]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)


Ahora se procede a importar Tensorflow para efecto de convertir en tensores los encodings generados en el paso anterior. Acá se junta cada uno a su correspondiente etiqueta. 

In [34]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

A continuación se importan los módulos de TFDistilBertForSequenceClassification que generalmente es usado para la tarea de análisis de sentimientos. También se importan los módulos *TFTrainer* y *TFTrainingArguments* que son los encargados de entrenar el modelo para hacer las nuevas predicciones. 


In [35]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

Hemos determinado el conjunto de argumentos que serán utilizados en la etapa de entrenamiento alojados en el  objeto *training_args*   y  ahora definiremos el modelo refiriendo el modelo preentrenado que vamos a utilizar, que en este caso es  _"distilbert-base-uncased"_, los argumentos los argumentos antes definidos   y los dos dataset (tensores de entrenamiento y prueba); para luego  entrenar el modelo que hemos definido.

In [36]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Ahora solo queda por aplicar  el entrenador que generamos al dataset de prueba, hacer la predicción, y la evaluación de las predicciones. Este procedimiento se encuentra definido en el [manual de fine-tuning](https://huggingface.co/transformers/v3.5.1/training.html) que tiene Hugging Face disponible. 

In [38]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.677699715750558}

In [39]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[-0.02710662, -0.0705149 ],
       [-0.06137016, -0.09839323],
       [ 0.00922669, -0.05698987],
       ...,
       [-0.01441754, -0.04300478],
       [-0.01974067, -0.06148365],
       [-0.0413641 , -0.09714585]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.6777530670166015})

In [40]:
trainer.predict(test_dataset)[1].shape

(1115,)

In [47]:
output=trainer.predict(test_dataset)[1]
output

array([0, 1, 0, ..., 0, 1, 0], dtype=int32)

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score

confusion_matrix=confusion_matrix(y_test,output)
confusion_matrix


array([[955,   0],
       [  0, 160]])

In [43]:
acc=accuracy_score(y_test,output)
acc

1.0

In [45]:
trainer.save_model('tf_model')

## Conclusiones

Se realiza el ejercicio de *fine-tuning* siguiendo cada una de los pasos que indica  HuggingFace en el  [manual de fine-tuning](https://huggingface.co/transformers/v3.5.1/training.html) y se  tiene una respuesta altamente satisfactoria del modelo con un accuracy perfecto, lo cual es viable en el entendido de que partimos de un  [modelo de fill-mask](https://huggingface.co/distilbert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.) entrenado con |un corpus enorme  y lo adaptamos a un modelo de clasificación binaria. 

este ejercicio a pesar de ser muy poco complejo como labor final si es muy satisfactorio a nivel personal por haber podido colocar en producción  la adaptación de un modelo preentrenado en una labor absolutamente diferente de la inicial. 

